# without any optimization

In [1]:
%load_ext line_profiler
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
nltk.download('stopwords')
nltk.download("wordnet")
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
MAX_LENGTH = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

def clean_sentence(sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    sentence = remove_emoji(sentence)
    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

def remove_stopwords(tokens, stopwords):
    clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
def lemmatize(tokens, lemma):
    lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens

# fix sentence length
def trunc_padding(sentence):
    modify_sentence = sentence.copy()
    if len(modify_sentence) >= MAX_LENGTH:
        modify_sentence = modify_sentence[:MAX_LENGTH]
    else:
        modify_sentence.extend(list(["0"] * (MAX_LENGTH - len(modify_sentence))))
    return modify_sentence

stopwords = nltk.corpus.stopwords.words('english')
lemma = WordNetLemmatizer()

def processing(df, stopwords, lemma):
    df['text'] = df['text'].apply(lambda sentence: sentence.lower())
    df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
    # tokenization
    df['text'] = df['text'].apply(lambda sentence: sentence.split())
    # remove stopwords
    df['text'] = df['text'].apply(lambda sentence: remove_stopwords(sentence, stopwords))
    # lemmalization 
    df['text'] = df['text'].apply(lambda sentence: lemmatize(sentence, lemma))
    # sentence length before padding
    df['length'] = df['text'].apply(lambda x: len(x))
    # fix sentence length
    df['text'] = df['text'].apply(lambda sentence: trunc_padding(sentence))
    # sentence length after padding
    df['length_padding'] = df['text'].apply(lambda x: len(x))

train= pd.read_csv('train.csv')
test= pd.read_csv("test.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
%lprun -f processing processing(train, stopwords, lemma)

Timer unit: 1e-07 s

Total time: 3.53028 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_6460\4085473941.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    78                                           
    79         1      50787.0  50787.0      0.1  
    80         1     921635.0 921635.0      2.6  
    81                                           
    82         1      89379.0  89379.0      0.3  
    83                                           
    84         1    1401477.0 1401477.0      4.0  
    85                                           
    86         1   32633575.0 32633575.0     92.4  
    87                                           
    88         1      32938.0  32938.0      0.1  
    89                                           
    90         1     141000.0 141000.0      0.4  
    91          

In [3]:
%lprun -f processing processing(test, stopwords, lemma)

Timer unit: 1e-07 s

Total time: 0.39762 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_6460\4085473941.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    78                                           
    79         1      42374.0  42374.0      1.1  
    80         1     373807.0 373807.0      9.4  
    81                                           
    82         1      75361.0  75361.0      1.9  
    83                                           
    84         1     672020.0 672020.0     16.9  
    85                                           
    86         1    2730069.0 2730069.0     68.7  
    87                                           
    88         1      17505.0  17505.0      0.4  
    89                                           
    90         1      49118.0  49118.0      1.2  
    91            

# First Itertool optimization

In [1]:
%load_ext line_profiler
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
nltk.download('stopwords')
nltk.download("wordnet")
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
MAX_LENGTH = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train= pd.read_csv('train.csv')
test= pd.read_csv("test.csv")

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import itertools

nltk.download('stopwords')
nltk.download('wordnet')

def remove_emoji(sentence):
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', sentence)
stopwords = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

# combine func clean_sentence,remove_stopwords,lemmatize 
def clean_sentence(sentence): 
    sentence = re.sub(r'http\S+', ' ', sentence) # remove urls
    sentence = remove_emoji(sentence) # remove emojis
    sentence = re.sub("[^0-9A-Za-z ]", "", sentence) # remove punctuations
    sentence = sentence.replace('  ',"") # remove double spaces
    sentence = sentence.lower().split() # lowercase and tokenize
    sentence = itertools.filterfalse(lambda x: x in stopwords, sentence) # remove stopwords
    sentence = list(itertools.chain.from_iterable(map(lambda x: [lemma.lemmatize(x, pos='v')], sentence))) # lemmatize
    return sentence

#fix sentence length
def trunc_padding(sentence):
    sentence = itertools.islice(itertools.chain(sentence, itertools.repeat('0')), 0, MAX_LENGTH) # fix sentence length
    return list(sentence)


def processing(df):
    df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
    df['length'] = df['text'].apply(lambda x: len(x))
    df['text'] = df['text'].apply(lambda sentence: trunc_padding(sentence))
    df['length_padding'] = MAX_LENGTH


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
%lprun -f processing processing(train)

Timer unit: 1e-07 s

Total time: 3.40684 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_12120\1775499896.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                           
    77         1   33876578.0 33876578.0     99.4  
    78         1      34907.0  34907.0      0.1  
    79         1     152833.0 152833.0      0.4  
    80         1       4034.0   4034.0      0.0

In [3]:
%lprun -f processing processing(test)

Timer unit: 1e-07 s

Total time: 0.363519 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_12120\1775499896.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                           
    77         1    3529557.0 3529557.0     97.1  
    78         1      18275.0  18275.0      0.5  
    79         1      83305.0  83305.0      2.3  
    80         1       4058.0   4058.0      0.1

# Second Itertool optimization

In [1]:
%load_ext line_profiler
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
nltk.download('stopwords')
nltk.download("wordnet")
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
MAX_LENGTH = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train= pd.read_csv('./train.csv')
test= pd.read_csv("./test.csv")
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import itertools

nltk.download('stopwords')
nltk.download('wordnet')


stopwords = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

# combine func clean_sentence,remove_stopwords,lemmatize 
def clean_sentence(sentence): 
    sentence = re.sub(r'http\S+', ' ', sentence) # remove urls
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r' ', sentence)
    sentence = re.sub("[^0-9A-Za-z ]", "", sentence) # remove punctuations
    sentence = sentence.replace('  ',"") # remove double spaces
    sentence = sentence.lower().split() # lowercase and tokenize
    sentence = (lemma.lemmatize(x, pos='v') for x in sentence if x not in stopwords) # lemmatize and remove stopwords using generator expression
    return list(sentence)


#fix sentence length
def trunc_padding(sentence):
    sentence = itertools.islice(itertools.chain(sentence, itertools.repeat('0')), 0, MAX_LENGTH) # fix sentence length
    return list(sentence)


def processing(df):
    df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    df['length'] = df['text'].map(lambda x: len(x))
    df['text'] = df['text'].map(lambda sentence: trunc_padding(sentence))
    df['length_padding'] = MAX_LENGTH



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
%lprun -f processing processing(train)

Timer unit: 1e-07 s

Total time: 3.36837 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_12136\2561017597.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    74                                           
    75         1   33512587.0 33512587.0     99.5  
    76         1      31706.0  31706.0      0.1  
    77         1     135410.0 135410.0      0.4  
    78         1       4010.0   4010.0      0.0

In [3]:
%lprun -f processing processing(test)

Timer unit: 1e-07 s

Total time: 0.34409 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_12136\2561017597.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
    74                                           
    75         1    3299331.0 3299331.0     95.9  
    76         1      28105.0  28105.0      0.8  
    77         1     108751.0 108751.0      3.2  
    78         1       4716.0   4716.0      0.1

Method 1: Reduce unnecessary function calls and merge multiple functions into a few. Python function calls are an expensive operation as they require passing parameters between functions and jumping to other parts of the code. Reducing unnecessary function calls can reduce this additional overhead and improve program performance.

Method 2: Using the map() function in the Pandas library can optimize Python program performance. When processing DataFrame objects, apply() is a common method. However, map() is usually faster than apply() because it does not call Python functions like apply() does. Instead, it utilizes built-in optimization techniques to process data.

Method 3: Using generator expressions can further improve program performance. Generator expressions are a concise and efficient way to handle lists or other iterable objects. Compared to using map() and filter() functions, generator expressions can calculate each value all at once, rather than loading the entire list into memory, reducing memory overhead and improving program performance.

# Second Itertool optimization with Cython

In [1]:
%load_ext Cython
%load_ext line_profiler

In [2]:
%%cython 
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1
# cython: profile=True
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import itertools
nltk.download('stopwords')
nltk.download("wordnet")
cdef seed_everything(int seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
cdef int MAX_LENGTH = 30
cdef object device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cdef clean_sentence(object sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    cdef object emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r' ', sentence)

    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

cdef remove_stopwords(list tokens, object stopwords):
    cdef list clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
cdef lemmatize(list tokens, object lemma):
    cdef list lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens

cdef trunc_padding(list sentence):
    sentence = list(itertools.islice(itertools.chain(sentence, itertools.repeat('0')), 0, MAX_LENGTH)) # fix sentence length
    return sentence

cdef object stopwords = nltk.corpus.stopwords.words('english')
cdef object lemma = WordNetLemmatizer()

def processing(df: pd.DataFrame, object stopwords, object lemma):
    df['text'] = df['text'].map(lambda sentence: sentence.lower())
    df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    # tokenization 123123
    df['text'] = df['text'].map(lambda sentence: sentence.split())
    # remove stopwords
    df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    # lemmalization 
    df['text'] = df['text'].map(lambda sentence: lemmatize(sentence, lemma))
    # sentence length before padding
    df['length'] = df['text'].map(lambda x: len(x))
    # fix sentence length
    df['text'] = df['text'].map(lambda sentence: trunc_padding(sentence))
    # sentence length after padding
    df['length_padding'] = df['text'].map(lambda x: len(x))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download("wordnet")
train= pd.read_csv('./train.csv')
test= pd.read_csv("./test.csv")
stopwords = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
%lprun -f processing processing(train, stopwords, lemma)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Timer unit: 1e-07 s

Total time: 3.25333 s
File: C:\Users\czy\.ipython\cython\_cython_magic_9b41f3ce78be3cc6a616579f23709c5e.pyx
Function: processing at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                           def processing(df: pd.DataFrame, object stopwords, object lemma):
    77         1      52813.0  52813.0      0.2      df['text'] = df['text'].map(lambda sentence: sentence.lower())
    78         1     847476.0 847476.0      2.6      df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    79                                               # tokenization 123123
    80         1      73099.0  73099.0      0.2      df['text'] = df['text'].map(lambda sentence: sentence.split())
    81                                               # remove stopwords
    82         1     135215.0 135215.0      0.4      df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    83              

In [4]:
%lprun -f processing processing(test, stopwords, lemma)


Timer unit: 1e-07 s

Total time: 0.319202 s
File: C:\Users\czy\.ipython\cython\_cython_magic_9b41f3ce78be3cc6a616579f23709c5e.pyx
Function: processing at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                           def processing(df: pd.DataFrame, object stopwords, object lemma):
    77         1      33561.0  33561.0      1.1      df['text'] = df['text'].map(lambda sentence: sentence.lower())
    78         1     297020.0 297020.0      9.3      df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    79                                               # tokenization 123123
    80         1      89781.0  89781.0      2.8      df['text'] = df['text'].map(lambda sentence: sentence.split())
    81                                               # remove stopwords
    82         1      39306.0  39306.0      1.2      df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    83             

without any optimization:<br>
Total time for processing training data: 3.53028 seconds.<br>
Total time for processing testing data: 0.39762 seconds.<br>

First Itertool optimization:<br>
Total time for processing training data: 3.40684 seconds, an improvement of about 3.5% relative to the previous time.<br>
Total time for processing testing data: 0.363519 seconds, an improvement of about 8.9% relative to the previous time.<br>

Second Itertool optimization:<br>
Total time for processing training data: 3.36837 seconds, an improvement of about 1.1% relative to the previous time.<br>
Total time for processing testing data: 0.34409 seconds, an improvement of about 5.3% relative to the previous time.<br>

Second Itertool optimization with Cython:<br>
Total time for processing training data: 3.25333 seconds, an improvement of about 3.4% relative to the previous time.<br>
Total time for processing testing data: 0.319202 seconds, an improvement of about 7.0% relative to the previous time.<br>

The percentage improvement relative to the original data can be calculated by comparing the total times for processing training and testing data between the original and final optimizations:

- Total time for processing training data improved from 3.53028 seconds to 3.25333 seconds, which is a percentage improvement of approximately (1 - 3.25333/3.53028) * 100% = 7.8%.
- Total time for processing testing data improved from 0.39762 seconds to 0.319202 seconds, which is a percentage improvement of approximately (1 - 0.319202/0.39762) * 100% = 19.7%.

Overall, the final optimization with Cython and the second Itertool optimization achieved a total percentage improvement of approximately 7.8% for processing training data and 19.7% for processing testing data relative to the original processing times.


